In [12]:
import torch
import numpy as np

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [13]:
%%capture
from transformers import AutoTokenizer, AutoModel, pipeline, OPTForCausalLM

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-125m")

# Load model with pre-trained head
model = OPTForCausalLM.from_pretrained("facebook/opt-125m", output_attentions=True, output_hidden_states=True)

generator = pipeline('text-generation', model="facebook/opt-125m")
#generator = pipeline('text-generation', model=model)


In [14]:
generator("Bush did 9/11 because")

[{'generated_text': "Bush did 9/11 because he was a terrorist.\nI'm not sure if you're being"}]

In [15]:
# for name, param in model.named_parameters():
for name, param in model.named_parameters():
    print(name)
    # print(param.shape)

# print(model.decoder.layers[0].self_attn)

model.decoder.embed_tokens.weight
model.decoder.embed_positions.weight
model.decoder.final_layer_norm.weight
model.decoder.final_layer_norm.bias
model.decoder.layers.0.self_attn.k_proj.weight
model.decoder.layers.0.self_attn.k_proj.bias
model.decoder.layers.0.self_attn.v_proj.weight
model.decoder.layers.0.self_attn.v_proj.bias
model.decoder.layers.0.self_attn.q_proj.weight
model.decoder.layers.0.self_attn.q_proj.bias
model.decoder.layers.0.self_attn.out_proj.weight
model.decoder.layers.0.self_attn.out_proj.bias
model.decoder.layers.0.self_attn_layer_norm.weight
model.decoder.layers.0.self_attn_layer_norm.bias
model.decoder.layers.0.fc1.weight
model.decoder.layers.0.fc1.bias
model.decoder.layers.0.fc2.weight
model.decoder.layers.0.fc2.bias
model.decoder.layers.0.final_layer_norm.weight
model.decoder.layers.0.final_layer_norm.bias
model.decoder.layers.1.self_attn.k_proj.weight
model.decoder.layers.1.self_attn.k_proj.bias
model.decoder.layers.1.self_attn.v_proj.weight
model.decoder.layers

In [16]:
import torch.nn.utils.prune 

# param_name is like "weight", tells which part of module to access for tensor
def init_mask(module, param_name):
    # mask=torch.ones_like(module.weight)
    # mask=torch.ones_like(getattr(module, param_name))
    mask=torch.zeros_like(getattr(module, param_name))

    mask_applier = torch.nn.utils.prune.CustomFromMask(mask)
    # apply the mask to the module
    mask_applier.apply(module=module, name=param_name, mask=mask)
    
# layer = model.get_decoder().layers[0].fc1
layer = model.get_decoder().layers[0].self_attn.out_proj

# the two actual tensors but in module form
weight_layer = layer.weight
bias_layer = layer.bias

# init_mask(layer, "weight")

# torch.nn.utils.prune.CustomFromMask(mask)

In [17]:
# A class to mask a transformer's layers
class Masker:

    # apply a mask to a module
    # def make_mask(module, param_name):
    
    # initialize the masks
    def __init__(self, model):
        # set up module dictionary, key is name and value is 
        module_dict = {}
        for n, m in model.named_modules():
            module_dict[n] = m
        
    # mask the whole model
    def mask(self, model, amount=.2):
        module_dict = {}
        for n, m in model.named_modules():
            module_dict[n] = m

        for n, m in model.named_parameters():
            module_name, param_type = self.get_module_name(n)
            # perform the masking
            module = module_dict[module_name]
            torch.nn.utils.prune.l1_unstructured(module=module, name=param_type, amount=.2)

        
    # apply desired mask (remove proportion of lowest weights)
    # name is "weight" or "bias"
    # prop is float between 0 and 1, higher prunes more values
    # def calculate_mask(self, module, name, prop):
    #     torch.nn.utils.prune.L1Unstructured.apply(module, name, prop)

    # get module name and parameter type from parameter name
    def get_module_name(self, param_name):
        if name[-5] == ".bias":
            return name[:-5], "bias"
        elif name[-7] == ".weight":
            return name[:-7, "weight"]
        
        # iterate through all 

In [40]:
def get_module_name(param_name):
    # print(param_name)
    # print(param_name[-7:])
    # print(param_name[-5:])
    if param_name[-5:] == ".bias":
        return param_name[:-5], "bias"
    elif param_name[-7:] == ".weight":
        return param_name[:-7], "weight"
    else:
        return None, None

# mask the whole model
def mask(model, amount=.2):
    module_dict = {}
    for n, m in model.named_modules():
        module_dict[n] = m
    # print(module_dict.keys())
    
    parameter_list = []
    for n, m in model.named_parameters():
        parameter_list.append(n)
    # print(parameter_list)

    for n in parameter_list:
        module_name, param_type = get_module_name(n)
        if module_name is None or param_type is None:
            continue
        # perform the masking
        module = module_dict[module_name]
        torch.nn.utils.prune.l1_unstructured(module=module, name=param_type, amount=.2)

In [41]:
input1 = tokenizer("Hello, my dog is cute", return_tensors="pt", padding="max_length", truncation=True)
input2 = tokenizer("What the fuck did you just fucking say about me, you little bitch?", return_tensors="pt", padding="max_length", truncation=True)
output = model.generate(input1.input_ids, max_length=300, num_return_sequences=1, temperature=0.5, top_p=0.95)
tokenizer.decode(output[0], skip_special_tokens=True)

KeyboardInterrupt: 

In [61]:
%%capture
from transformers import AutoTokenizer, AutoModel, pipeline, OPTForCausalLM

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained("facebook/opt-125m")

# Load model with pre-trained head
model = OPTForCausalLM.from_pretrained("facebook/opt-125m", output_attentions=True, output_hidden_states=True)

generator = pipeline('text-generation', model="facebook/opt-125m")
#generator = pipeline('text-generation', model=model)


In [62]:
mask(model=model, amount=.0)

In [63]:
input1 = tokenizer("Hello, my dog is cute", return_tensors="pt", padding="max_length", truncation=True)
input2 = tokenizer("What the fuck did you just fucking say about me, you little bitch?", return_tensors="pt", padding="max_length", truncation=True)
output = model.generate(input1.input_ids, max_length=40, num_return_sequences=1, temperature=0.5, top_p=0.95)
tokenizer.decode(output[0], skip_special_tokens=True)

'Hello, my dog is cute.\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n'